In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from keras.optimizers import Adam
import os
from IPython.display import display
import json
from os.path import exists
from keras.applications import MobileNetV3Small

from numpy.random import choice as choose

import sys
sys.path.insert(0, "../Src/")
import basicUtil
import transferUtil

%autosave 5

Autosaving every 5 seconds


Autosaving every 5 seconds


Autosaving every 5 seconds


In [2]:
imageShape = (180,180)
inputShape = [imageShape[0],imageShape[1],3]
train = transferUtil.loadDataWithResizeShape("training",imageShape)
dev,test = transferUtil.loadDataWithResizeShape("testing",imageShape,.5)

base_model = MobileNetV3Small(
    include_top=False,
    classes=len(train.class_names),
    classifier_activation=None,
    input_shape=inputShape
)
for layer in base_model.layers:
    layer.trainable = False

possibleNeuronsPerLayer = basicUtil.createRangeFromMidpoint(16,32)
possibleEpochs = basicUtil.createRangeFromMidpoint(5,10)

adamLearningRateCriticalPoints = (1e-4,1e-2)
L2CriticalPoints = (1e-2,1e3) 

n_NeuronsPerFCLayers = []
n_Epochs = []
adamLearningRates = []
L2Rates = []
trainScores = []
devScores = []

trial = 0
bestDevScore = 0
oversizedNeuronNumbers = []
oversizedEpochNumbers = []

while trial < 100:
    numberOfNeuronsPerFCLayer = choose(possibleNeuronsPerLayer)
    numberOfEpochs = choose(possibleEpochs)
    
    adamLearningRate = basicUtil.generateAdamLearningRate(adamLearningRateCriticalPoints[0],adamLearningRateCriticalPoints[1])
    L2Rate = basicUtil.generateL2(L2CriticalPoints[0],L2CriticalPoints[1])
    model = transferUtil.createTransferableModel(base_model, numberOfNeuronsPerFCLayer, adamLearningRate, L2Rate)
    
    model.fit(train,epochs=numberOfEpochs,verbose=0)

    model_path = f'../Models/MobileNetTrials/mobilenet_model_{trial}.h5'
    model.save(model_path)
    model_size = os.path.getsize(model_path) / (1024 * 1024)
    if model_size < 40:
        oversizedNeuronNumbers = []
        oversizedEpochNumbers = []
        print('trainScore')
        trainScore = model.evaluate(train)[1]
        print('devScore')
        devScore = model.evaluate(dev)[1]

        if (devScore > 0.91) and (devScore > bestDevScore):
            testScore = model.evaluate(test)[1]
            model_path = f'../Models/best_mobilenet_model_.h5'
            model.save(model_path)
            bestModelParams = {
                'n_NeuronsPerFCLayers' : int(numberOfNeuronsPerFCLayer),
                'n_Epochs' : int(numberOfEpochs),
                'adamLearningRates' : adamLearningRate,
                'L2Rates' : L2Rate,
                'modelSize' : model_size,
                'trainScore': trainScore,
                'devScore': devScore,
                'testScore': testScore
            }
            with open('../Models/best_mobilenet_model_params.json', 'w') as f:
                json.dump(bestModelParams, f)
            bestDevScore = devScore       

        n_NeuronsPerFCLayers.append(numberOfNeuronsPerFCLayer)
        n_Epochs.append(numberOfEpochs)
        
        adamLearningRates.append(adamLearningRate)
        L2Rates.append(L2Rate)
        trainScores.append(trainScore)
        devScores.append(devScore)
        
        print('concluding trial ',trial)
        trial += 1
    else:
        print(f'redoing trial {trial}. Model was {model_size}MB.')
        failedTrial = transferUtil.createTransferModelParametersDF([numberOfNeuronsPerFCLayer],[numberOfEpochs],[adamLearningRate],
                                                                   [L2Rate],[np.nan],[np.nan])
        display(failedTrial)
        oversizedNeuronNumbers.append(numberOfNeuronsPerFCLayer)
        oversizedEpochNumbers.append(numberOfEpochs)
        if len(oversizedNeuronNumbers) >= 3:
            possibleNeuronsPerLayer = [x for x in possibleNeuronsPerLayer if x < np.mean(oversizedNeuronNumbers)]
            possibleEpochs = [x for x in possibleEpochs if x < np.mean(oversizedEpochNumbers)]
            oversizedNeuronNumbers = []
            oversizedEpochNumbers = []
        
    if (trial % 10 == 9): 
        modelParametersDF = transferUtil.createTransferModelParametersDF(n_NeuronsPerFCLayers,n_Epochs,adamLearningRates,
                                                                         L2Rates,trainScores,devScores)
        modelParametersDF = modelParametersDF.sort_values(by='trainScore', ascending=False)
        display(modelParametersDF)
        
        top5 = modelParametersDF[0:5]
        possibleNumberOfNeuronsPerFCLayer = basicUtil.getAdjustedRange(top5['n_NeuronsPerFCLayers'])
    
        possibleNumberOfEpochs = basicUtil.getAdjustedRange(top5['n_Epochs'])
        adamLearningRateCriticalPoints = basicUtil.calculateLogisticCriticalPoints(top5['adamLearningRates'])
        L2CriticalPoints = basicUtil.calculateLogisticCriticalPoints(top5['L2Rates'])

        n_NeuronsPerFCLayers = []
        n_Epochs = []
        adamLearningRates = []
        L2Rates = []
        trainScores = []
        devScores = []

        if bestDevScore > 0.91:
            trial = 101
            
basicUtil.displayFinalResults('../Models/best_mobilenet_model_params.json')
    

In non split
Found 2392 files belonging to 2 classes.
In split
Found 597 files belonging to 2 classes.
Using 299 files for training.
Using 298 files for validation.


2024-09-22 19:25:57.616572: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-09-22 19:25:57.616944: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2392]
	 [[{{node Placeholder/_0}}]]
2024-09-22 19:26:02.557625: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


trainScore
75/75 [==============================] - 32s 386ms/step - loss: 2.7779 - f1_score: 0.6466
devScore


2024-09-22 19:27:05.016528: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]
2024-09-22 19:27:05.016845: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]


10/10 [==============================] - 5s 307ms/step - loss: 2.7912 - f1_score: 0.6354
concluding trial  0
trainScore
75/75 [==============================] - 36s 422ms/step - loss: 0.5401 - f1_score: 0.8848
devScore
10/10 [==============================] - 6s 361ms/step - loss: 0.6247 - f1_score: 0.8173
concluding trial  1
trainScore
75/75 [==============================] - 35s 409ms/step - loss: 0.4674 - f1_score: 0.9074
devScore
10/10 [==============================] - 6s 343ms/step - loss: 0.5239 - f1_score: 0.8672
concluding trial  2
trainScore
75/75 [==============================] - 24s 262ms/step - loss: 0.6568 - f1_score: 0.9976
devScore
10/10 [==============================] - 3s 187ms/step - loss: 0.6997 - f1_score: 0.9892


2024-09-22 19:35:55.987738: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [298]
	 [[{{node Placeholder/_4}}]]
2024-09-22 19:35:55.987878: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [298]
	 [[{{node Placeholder/_4}}]]


10/10 [==============================] - 3s 251ms/step - loss: 0.6815 - f1_score: 0.9913
concluding trial  3
trainScore
75/75 [==============================] - 19s 227ms/step - loss: 0.7912 - f1_score: 0.9697
devScore
10/10 [==============================] - 3s 152ms/step - loss: 0.8329 - f1_score: 0.9714
concluding trial  4
trainScore
75/75 [==============================] - 17s 202ms/step - loss: 0.2575 - f1_score: 0.9889
devScore
10/10 [==============================] - 2s 142ms/step - loss: 0.3191 - f1_score: 0.9743
concluding trial  5
trainScore
75/75 [==============================] - 18s 214ms/step - loss: 0.5639 - f1_score: 0.8052
devScore
10/10 [==============================] - 3s 187ms/step - loss: 0.5904 - f1_score: 0.7733
concluding trial  6
trainScore
75/75 [==============================] - 18s 222ms/step - loss: 0.6954 - f1_score: 0.5971
devScore
10/10 [==============================] - 2s 159ms/step - loss: 0.7281 - f1_score: 0.6047
concluding trial  7
trainScore
75/7

,n_NeuronsPerFCLayers,n_Epochs,adamLearningRates,L2Rates,trainScore,devScore
3,9,3,0.000104,0.232976,0.997602,0.989181
5,20,4,0.000156,1.537194,0.988888,0.974289
4,7,3,0.000108,2.455295,0.969735,0.971396
2,4,1,0.001130,2.091349,0.907359,0.867212
1,26,8,0.003604,0.174788,0.884776,0.817288
6,2,7,0.000240,1.718013,0.805222,0.773307
8,31,4,0.000350,281.658269,0.740419,0.727612
0,17,1,0.007355,54.529539,0.646564,0.635401
7,12,5,0.000434,37.331961,0.597056,0.604668


Final Model


,n_NeuronsPerFCLayers,n_Epochs,adamLearningRates,L2Rates,modelSize,trainScore,devScore,testScore
0,9,3,0.000104,0.232976,6.015419,0.997602,0.989181,0.991311
